In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
from os import environ, getenv, path
from typing import TypedDict, Dict, Union, List
from simulator_funcs import (
    simulate,
    load_ship,
    get_routing_table_from_mmsi,
    plot_df,
    get_routing_and_emissions,
    emissions_multiple_vessels,
)

import warnings

warnings.filterwarnings("ignore")

from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.lookup_ship_data import (
    fetch_ship_data,
)
from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.database import (
    get_connection_pool,
    get_engine,
)
import typing

## Color code explanation
<p style="color:#FE774C; font-size:12pt;"> Red means that values <i>have</i> to be inserted.</p> 
<p style="color:#9D59F4; font-size:12pt"> Purple means that values <i>can</i> be inserted if other that the default values are needed.</p> 


<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert all the mmsis for the vessels interest in here </p> 
<p> For example like the list under with two vessels (change the mmsis and add more of them if needed). </p> 

In [2]:
mmsis = [212552000, 319009700]

<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert the coordinates of the ports in the simulated paths </p> 
<p> For example like the list under where the path will go from Oslo to Kiel to Amsterdam. </p> 

In [3]:
coordinates = [[10.736223, 59.904479], [10.143301, 54.320150], [4.8079, 52.412]]

# Specifying properties needed for routing

In the following cells we are specifying speed and draught for the vessels as well as graphing, time resolution, how detailed the routing should be and how much other vessel data should be weighed for the routing. 

<p style="color:#9D59F4; font-size:15pt; font-weight:bold"> Make a list for the speeds to simulate with for each vessel  </p> 
<p> If the speed is set to None, the maximum speed for the vessel is chosen. </p> 

In [4]:
speeds = [None] * len(mmsis)
# Or:
# speeds = [10, 10] #Here, the speed for both vessels is set to 10 knots
speeds

[None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> List of draught for the different vessels </p> 
<p> If they are set to None, the draught will be found in the data about the vessel particulars. </p> 

In [5]:
draughts = [None] * len(mmsis)
# Or:
# draughts = [7, 9] #here the draught for the first ship is 7m and for the second ship it is 9 m
draughts

[None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose grapfing types for the vessels </p> 
<p> This is also found from the vessel particulars, but in the incidences where it cannot be found, it needs to be specified. Possible graph types are passenger, fishing, cargo, tanker and tug. </p> 

In [6]:
graphs = [None] * len(mmsis)
# Or:
# graphs = ['cargo', 'tug']
graphs

[None, None]

### Make a dataframe of the mmsis, speeds, draughts and graphs
This is used as input in the function finding routes and emissions.

In [7]:
zipped_lists = list(zip(speeds, draughts, graphs))
df_input_properties = pd.DataFrame(
    zipped_lists, index=mmsis, columns=["speeds", "draughts", "graphs"]
)

In [8]:
df_input_properties

,speeds,draughts,graphs
212552000,None,None,None
319009700,None,None,None


<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose the time resolution for the graphing </p> 
<p> The default value is every 60th minute. Choose a value between 5 and 240. </p> 

In [9]:
time_resolution = 60

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Decide how fine/coarse the routing should be </p> 

 * 0 = Coarse (fast)
 * 1 = Coarse + refined
 * 2 = Fine (slow)

In [10]:
detailed_routing = 0

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> How much should popular routes be weighted in the routing? </p> 
<p> Choose a value between 0 and 0.95. Default is 0.3. </p> 

In [11]:
cost_density = 0.3

## Finding paths and emissions for each vessel

Using a premade function called emissions_multiple_vessels that takes the coordinates chosen and the df_input_properties as arguments. OBS, this might take a while. 

In [12]:
%%time
df_summed_emissions, list_of_df_emissions, list_of_paths = emissions_multiple_vessels(
    coordinates,
    df_input_properties,
    None,
    None,
    time_resolution,
    detailed_routing,
    cost_density,
)

CPU times: user 15.4 s, sys: 505 ms, total: 15.9 s
Wall time: 23.9 s


### Summed emissions for all vessels compared

The vessel at the top is the one that emitts the least amount of CO2.

In [13]:
df_summed_emissions

,emissions_CO2,emissions_CO,emissions_SOX,emissions_N2O,emissions_NOX,emissions_PM,emissions_CH4
ssvid,,,,,,,
319009700.0,5.591187e+07,47965.509719,48406.604353,2563.454380,6.498691e+05,17247.655382,902.123654
212552000.0,8.570677e+07,71672.791683,90470.236174,3912.549372,1.725141e+06,29074.798519,1338.138231


## Show the paths
This one is for the vessel that uses the least amount of CO2

In [14]:
min_co2_mmsi = int(df_summed_emissions.index[0])
min_co2_mmsi

319009700

In [15]:
list_of_paths[mmsis.index(min_co2_mmsi)]

{"initialViewState": {"latitude": 57.2, "longitude": 10, "zoom": 3}, "layers": [{"@@type": "PathLayer", "data": [{"path": [[10.72265625, 59.853515625], [10.72265625, 59.853515625], [10.72265625, 59.677734375], [10.72265625, 59.677734375], [10.72265625, 59.501953125], [10.72265625, 59.501953125], [10.72265625, 59.326171875], [10.72265625, 59.23828125], [10.72265625, 59.150390625], [10.72265625, 58.974609375], [10.72265625, 58.740234375], [10.72265625, 58.53515625], [10.8984375, 58.359375], [11.07421875, 58.18359375], [11.07421875, 58.0078125], [11.25, 57.83203125], [11.42578125, 57.626953125], [11.42578125, 57.392578125], [11.77734375, 57.216796875], [11.77734375, 57.041015625], [11.77734375, 56.806640625], [11.77734375, 56.6015625], [11.77734375, 56.513671875], [11.42578125, 56.337890625], [11.42578125, 56.162109375], [11.42578125, 55.986328125], [11.42578125, 55.810546875], [11.07421875, 55.634765625], [11.07421875, 55.634765625], [11.07421875, 55.458984375], [11.07421875, 55.37109375], [11.07421875, 55.283203125], [11.07421875, 55.107421875], [11.07421875, 55.107421875], [10.72265625, 54.931640625], [10.72265625, 54.931640625], [10.72265625, 54.755859375], [10.72265625, 54.580078125], [10.72265625, 54.580078125], [10.37109375, 54.404296875], [10.37109375, 54.404296875], [10.01953125, 54.404296875], [9.66796875, 54.228515625], [9.66796875, 54.228515625], [9.31640625, 54.052734375], [9.31640625, 54.052734375], [8.96484375, 54.052734375], [8.96484375, 54.052734375], [8.61328125, 54.052734375], [8.4375, 54.052734375], [8.0859375, 54.052734375], [7.91015625, 54.052734375], [7.55859375, 53.876953125], [7.55859375, 53.876953125], [7.20703125, 53.876953125], [7.03125, 53.876953125], [6.6796875, 53.876953125], [6.328125, 53.876953125], [6.15234375, 53.876953125], [5.80078125, 53.701171875], [5.80078125, 53.701171875], [5.44921875, 53.525390625], [5.2734375, 53.525390625], [4.921875, 53.4375], [4.74609375, 53.349609375], [4.39453125, 53.173828125], [4.04296875, 52.998046875], [4.04296875, 52.763671875], [4.04296875, 52.646484375], [4.39453125, 52.470703125], [4.39453125, 52.470703125], [4.74609375, 52.470703125], [4.74609375, 52.470703125]]}], "getColor": "@@=[0,0,0, 150]", "id": "1b4a4a46-5a4f-492c-984d-0f25a1d60cd5", "widthMinPixels": 2}, {"@@type": "PointCloudLayer", "autoHighlight": true, "data": [{"distance_from_port_m": 5717.219100297454, "distance_from_shore_m": 2128.5882735590944, "draught_recent_m": 3.15, "emissions_CH4": 6.122379010660412, "emissions_CO": 277.3430984124586, "emissions_CO2": 180625.6076424588, "emissions_N2O": 9.859474061469541, "emissions_NOX": 2807.119740167983, "emissions_PM": 75.04578652887578, "emissions_SOX": 156.0896601158078, "in_a_eca": 1, "in_a_river": 0, "lat": 59.853515625, "lon": 10.72265625, "speed_knots": 5.0, "ssvid": 319009700.0, "timestamp": "2022-01-01 01:00:00", "total_power_demand_kw": 266.9994037119435}, {"distance_from_port_m": 5717.219100297454, "distance_from_shore_m": 2128.5882735590944, "draught_recent_m": 3.15, "emissions_CH4": 6.122379010660412, "emissions_CO": 277.3430984124586, "emissions_CO2": 180625.6076424588, "emissions_N2O": 9.859474061469541, "emissions_NOX": 2807.119740167983, "emissions_PM": 75.04578652887578, "emissions_SOX": 156.0896601158078, "in_a_eca": 1, "in_a_river": 0, "lat": 59.853515625, "lon": 10.72265625, "speed_knots": 5.0, "ssvid": 319009700.0, "timestamp": "2022-01-01 02:00:00", "total_power_demand_kw": 266.9994037119435}, {"distance_from_port_m": 25224.310248467875, "distance_from_shore_m": 0.0, "draught_recent_m": 3.15, "emissions_CH4": 6.122379010660412, "emissions_CO": 277.3430984124586, "emissions_CO2": 180625.6076424588, "emissions_N2O": 9.859474061469541, "emissions_NOX": 2807.119740167983, "emissions_PM": 75.04578652887578, "emissions_SOX": 156.0896601158078, "in_a_eca": 1, "in_a_river": 0, "lat": 59.677734375, "lon": 10.72265625, "speed_knots": 5.0, "ssvid": 319009700.0, "timestamp": "2022-01-01 03:00:00", "total_power_demand_kw": 266.999403711943

If you want to see the paths of the other vessels, uncomment the lines underneath. If there are more than two vessels you can also change the index to see the other (OBS, zero indexed). 

In [16]:
# max_co2_mmsi = int(df_summed_emissions.index[1])
# list_of_paths[mmsis.index(max_co2_mmsi)]